In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from tqdm import tqdm
import googlemaps
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
df = pd.read_csv("../data/final_naver_review.csv")
df.head()

,Unnamed: 0,시설명,리뷰
0,0,24시 강서 젠틀리동물의료센터,뼈다귀 맛집
1,1,24시 강서 젠틀리동물의료센터,굿
2,2,24시 강서 젠틀리동물의료센터,애기아파서 들렀는데 친절하게 잘 상담해주셨어요
3,3,24시 강서 젠틀리동물의료센터,야간에도 진료를 잘봐줘요
4,4,24시 강서 젠틀리동물의료센터,좋아요


In [6]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)
df.head()

,시설명,리뷰
0,24시 강서 젠틀리동물의료센터,뼈다귀 맛집
1,24시 강서 젠틀리동물의료센터,굿
2,24시 강서 젠틀리동물의료센터,애기아파서 들렀는데 친절하게 잘 상담해주셨어요
3,24시 강서 젠틀리동물의료센터,야간에도 진료를 잘봐줘요
4,24시 강서 젠틀리동물의료센터,좋아요


In [7]:
review_list = df["시설명"].unique().tolist()
review_list

['24시 강서 젠틀리동물의료센터',
 '24시 굿케어동물의료센터',
 '24시 글로리 동물병원',
 '24시 산들산들 동물병원',
 '24시 서울 탑 동물병원',
 '24시 시유동물메디컬센터',
 '24시 신도림S동물의료센터',
 '24시 안박동물병원',
 '24시 애니동물병원',
 '24시 청담우리 동물병원',
 '24시 SNC 동물메디컬센터',
 '24시루시드동물메디컬센터',
 '24시센트럴동물메디컬센터',
 '24시스마트동물메디컬센터',
 '24시아프리카동물메디컬센터',
 '24시연동물의료센터',
 '24시예담동물의료센터',
 '24시잠실ON동물의료센터',
 '24시OK동물병원',
 '25시동물병원',
 '25시동물의료센터',
 '331멍발소',
 '365 마음동물병원',
 '365동물병원',
 '38도씨식당',
 '가나동물병원(서울 강동)',
 '가나동물병원(서울 도봉구)',
 '가나종합동물병원',
 '가든펫동물병원',
 '가람동물병원(서울 강동)',
 '가로수동물병원',
 '가양쿨펫동물병원',
 '가온동물병원',
 '가원펫애견샵',
 '가재울봉쥬르동물병원',
 '가좌동물병원',
 '감성있개',
 '강남25동물병원',
 '강남동물병원',
 '강남펫스타일',
 '강동24시 SKY 동물의료센터',
 '강동동물병원',
 '강북 24시 N 동물의료센터',
 '강북동물병원',
 '강서YD동물의료센터',
 '강아지고양이편의점 고척점',
 '강아지고양이편의점 남가좌점',
 '강아지고양이편의점 목동점',
 '강아지고양이편의점 삼전점',
 '강아지고양이편의점 상계점',
 '강아지고양이편의점 중랑천점',
 '강아지고양이편의점 한성대점',
 '강아지놀이터',
 '강아지대학로',
 '강아지분양 미니몽 은평점',
 '강아지세상',
 '강아지와고양이나라 불광점',
 '강아지정류장',
 '강아지집',
 '강앤정 동물병원',
 '개과천선',
 '개님살롱 레이크사이드',
 '개라다이스',
 '개명동물병원',
 '개밥',
 '개밥주는고양이 신길점',
 '개밥파는고양이 까치산역점',


In [8]:
conn = mysql.connector.connect(
    host = " database-2.c3iym8yog7ht.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "pethub",
    password = "addinedu5",
    database = "pethub"
)

In [9]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM pet_infra")
result = cursor.fetchall()

In [12]:
df = pd.DataFrame(result)
df.rename(columns={0: "id", 1: "region_id", 2: "category_id", 3: "name", 4: "address", 5: "lat", 6: "lng"}, inplace=True)
df.head()

,id,region_id,category_id,name,address,lat,lng
0,1,12,6,은평구 반려견 놀이터,서울 은평구 진광동 42,37.4906,126.919
1,2,12,6,어린이대공원,광진구 능동로 216(구의문주차장 옆),37.5494,127.082
2,3,12,6,월드컵공원,마포구 증산로 32(평화의 공원 주차장 옆),37.5639,126.894
3,4,12,6,보라매공원,동작구 여의대방로20마길 11(보라매공원 향기원 옆),37.4927,126.919
4,5,12,6,매헌시민의숲,서울 서초구 매헌로 99 (양재천교 인근),37.4707,127.036


In [14]:
def getGoogleMapPlaceId(gmap, search_data):
    geocode = gmap.geocode(address=search_data)
    return geocode[0]['place_id']

In [15]:
def getGoogleMapReviews(gmap, place_id):
    try:
        detail = gmap.place(place_id, language="ko")
    except:
        detail = {"result": []}
    return detail['result']

In [10]:
google_map_key = os.getenv("google_map_key")
gmap = googlemaps.Client(key=google_map_key)

In [50]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    search_data = str(row["name"]) + " " + str(row["address"])
    geocode = gmap.geocode(search_data)
    df.loc[i, "lat"] = geocode[0].get("geometry")["location"]["lat"]
    df.loc[i, "lng"] = geocode[0].get("geometry")["location"]["lng"]
df.head()

100%|██████████| 3402/3402 [08:01<00:00,  7.06it/s]


,id,region_id,category_id,name,address,lat,lng
0,1,12,6,은평구 반려견 놀이터,서울 은평구 진광동 42,37.490619,126.919203
1,2,12,6,어린이대공원,광진구 능동로 216(구의문주차장 옆),37.549363,127.081813
2,3,12,6,월드컵공원,마포구 증산로 32(평화의 공원 주차장 옆),37.563873,126.893501
3,4,12,6,보라매공원,동작구 여의대방로20마길 11(보라매공원 향기원 옆),37.492678,126.919139
4,5,12,6,매헌시민의숲,서울 서초구 매헌로 99 (양재천교 인근),37.470685,127.0356


In [51]:
sql = "UPDATE pet_infra SET lat = %s, lng = %s WHERE id = %s"
for i, row in df.iterrows():
    cursor.execute(sql, (row["lat"], row["lng"], row["id"]))
    print((row["lat"], row["lng"], row["id"]))
    conn.commit()

(37.4906193, 126.9192028, 1)
(37.549363, 127.0818126, 2)
(37.5638735, 126.8935007, 3)
(37.4926778, 126.919139, 4)
(37.4706853, 127.0356004, 5)
(37.6477697, 127.0402485, 6)
(37.4968234, 127.10012, 7)
(37.5211564, 126.8805509, 8)
(37.5071197, 126.8744887, 9)
(37.580646, 127.0773483, 10)
(37.6214313, 127.0406246, 11)
(37.5515917, 127.0679532, 12)
(37.4516683, 126.8951977, 13)
(37.5629157, 126.8405143, 14)
(37.5554718, 126.8369299, 15)
(37.5080943, 127.1165836, 16)
(37.6196405, 126.9177319, 17)
(37.5602761, 127.1681898, 18)
(37.5416888, 127.1337301, 19)
(37.5114571, 126.9156161, 20)
(37.5473276, 126.8368214, 21)
(37.5250773, 126.967788, 22)
(37.682633, 127.045487, 23)
(37.4906406, 127.0812097, 24)
(37.6122908, 126.9317226, 25)
(37.5174938, 126.8654977, 26)
(37.505066, 126.9423567, 27)
(37.4796737, 126.9821564, 28)
(37.6106647, 127.0773901, 29)
(37.599235, 126.9589611, 30)
(37.5745873, 126.8971771, 31)
(37.5444748, 126.8708956, 32)
(37.5659043, 127.0033344, 33)
(37.5251955, 127.052372, 34)


In [16]:
infra_reviews = []
for i, row in df.iterrows():
    if row["name"] in review_list:
        continue
    search_data = str(row["name"]) + " " + str(row["address"])
    place_id = getGoogleMapPlaceId(gmap, search_data)
    reviews = getGoogleMapReviews(gmap, place_id)
    if 'reviews' in reviews:
        for review in reviews['reviews']:
           review_dict = {}
           review_dict["시설명"] = row["name"]
           review_dict["리뷰"] = review["text"]
           infra_reviews.append(review_dict)

print(len(infra_reviews), infra_reviews)

2503 [{'시설명': '은평구 반려견 놀이터', '리뷰': '자주 놀러가는 곳! 전반적으로 넓은 편이어서 좋아요. 봄에는 벚꽃도 예쁜 곳이예요'}, {'시설명': '은평구 반려견 놀이터', '리뷰': '그늘이 있어서 쉴수도 있고 대형견 소형견들 분리가 되어 있어 안전합니다\n강아지들이 많아 사회성 키우기에도 좋고 항상 활성화되어 있어 좋습니다'}, {'시설명': '은평구 반려견 놀이터', '리뷰': '크기는 보라매공원 사이즈에 비해 생각보다 아담한 듯. 깨끗하고 좋음.'}, {'시설명': '은평구 반려견 놀이터', '리뷰': '강아지가 근처만 가도 놀이터로 돌진합니다 ㅋㅋㅋㅋㅋ'}, {'시설명': '은평구 반려견 놀이터', '리뷰': '다른 강아지들하고 교류하며 사회성 기르기 좋아요. 여기 들어오는 강아지들은 비교적 다른 강아지들하고 잘 지내는 것 같아요. 소형견/대형견 나눠져 있어 체구에 맞게 어울려 놀 수 있어요.'}, {'시설명': '어린이대공원', '리뷰': '도심속의 넓은 공원으로 개설된지 수십년이 지나 나무들도 크고 동물원, 식물원, 지금도 명칭이 동일한지 모르지만 청룡열차를 비롯한 놀이기구 등이 여전하다. 주변에 사는 사람들에게는 쉽게 접근할 수 있으니 그야말로 소중한 공원이겠다.'}, {'시설명': '어린이대공원', '리뷰': '주차는 동물원인지 정문말고 다른데 했다. 가족 단위 방문객이 많아서 유모차 끌고다니기 편하다. 오랜만에 동물 봐서 재밌다. 동물원 설명도 아이들 수준에 맞춰져있어서 좋았다.'}, {'시설명': '어린이대공원', '리뷰': '금요일에 갔는데 줄 안서고 이용했어요 직원분들도 다 친절하시고 너무 무섭다 싶은 놀이기구는 없어서 잘 용했습니당 ^^( 회전그네는 생각보다 많이 어지러워요 ) 어릴때 방문했다가 성인되고 방문하니 옛추억도 생각나고 좋았습니당 동물원이랑 식물원은 무료관람 가능해요! 방문일자 (2024년 11월 1일)'}, {'시설명': '어린이대공원', '리뷰': '서울 도심 속에 자리잡은 오랜 역사를 자랑하는 유원지

In [17]:
review_df = pd.DataFrame(infra_reviews)
review_df.head()

,시설명,리뷰
0,은평구 반려견 놀이터,자주 놀러가는 곳! 전반적으로 넓은 편이어서 좋아요. 봄에는 벚꽃도 예쁜 곳이예요
1,은평구 반려견 놀이터,그늘이 있어서 쉴수도 있고 대형견 소형견들 분리가 되어 있어 안전합니다\n강아지들이...
2,은평구 반려견 놀이터,크기는 보라매공원 사이즈에 비해 생각보다 아담한 듯. 깨끗하고 좋음.
3,은평구 반려견 놀이터,강아지가 근처만 가도 놀이터로 돌진합니다 ㅋㅋㅋㅋㅋ
4,은평구 반려견 놀이터,다른 강아지들하고 교류하며 사회성 기르기 좋아요. 여기 들어오는 강아지들은 비교적 ...


In [24]:
cursor.execute("SELECT * FROM pet_infra")
result = cursor.fetchall()
infra_list = []
for data in result:
    infra_list.append(data[3])
infra_list

['은평구 반려견 놀이터',
 '어린이대공원',
 '월드컵공원',
 '보라매공원',
 '매헌시민의숲',
 '초안산근린공원',
 '송파구 반려견 놀이터',
 '영등포구 반려견 놀이터',
 '구로구 반려견 놀이터',
 '동대문구 반려견 놀이터',
 '북서울꿈의숲',
 '성동구 반려견 놀이터',
 '금천구 반려견 놀이터',
 '차지우동물병원',
 '박동물병원',
 '페토피아종합동물병원',
 '도그원동물의료센터',
 '피카소동물병원',
 '베베동물병원',
 '우신종합동물병원',
 '강서 더힐 동물병원',
 '에스동물병원',
 '도담동물병원',
 '서경석동물병원',
 '동물병원 움',
 '공원옆동물의료센터',
 '유케어종합동물병원',
 'VIP동물의료센터 서초동작점',
 '홍상희동물병원',
 '올리브동물병원',
 '상암 진 동물의료센터',
 '대현동물병원',
 '서울동물영상종양센터',
 '청담 고운 피부과 동물병원',
 '24시 딜라이트 동물의료센터',
 '동물병원 비아츠',
 '펫사랑동물병원',
 '아이원 동물병원',
 '더조은 동물의료센터',
 '하니동물병원',
 '포포동물병원',
 '율동물의료센터',
 '우신동물병원',
 '케어덴 동물치과&내과병원',
 '스마트동물병원',
 '바로슬동물병원',
 '청담 장튼튼내과동물병원',
 '신길S동물의료센터',
 '드림동물병원',
 '삼성펫클리닉',
 '24시예담동물의료센터',
 '청화종합동물병원',
 '동물병원알루',
 '스무해 동물병원',
 '중랑차동물병원',
 '진솔동물병원',
 '동물진료법인 포레스트',
 '피닉스동물병원',
 '에코동물병원',
 '헬로우동물병원 여의도점',
 '혜원동물병원',
 '스무살의꿈 동물병원',
 '나비셀 동물의료센터',
 '우리동물메디컬센터',
 '24시 수유 바른펫 동물의료센터',
 '쭈니동물병원',
 '에스디(SD)동물영상센터,서울',
 '힐링동물병원 &  건강검진센터',
 '이로동물의료센터',
 '아프지말게 동물병원',
 '엘 동물병원',
 '힐스타동물병원',
 '파파동물병원',
 '러브

In [25]:
review_df["시설명"] = [infra_list.index(each)+1 for each in tqdm(review_df["시설명"])]
review_df.head()

100%|██████████| 2503/2503 [00:00<00:00, 69014.47it/s]


,시설명,리뷰
0,1,자주 놀러가는 곳! 전반적으로 넓은 편이어서 좋아요. 봄에는 벚꽃도 예쁜 곳이예요
1,1,그늘이 있어서 쉴수도 있고 대형견 소형견들 분리가 되어 있어 안전합니다\n강아지들이...
2,1,크기는 보라매공원 사이즈에 비해 생각보다 아담한 듯. 깨끗하고 좋음.
3,1,강아지가 근처만 가도 놀이터로 돌진합니다 ㅋㅋㅋㅋㅋ
4,1,다른 강아지들하고 교류하며 사회성 기르기 좋아요. 여기 들어오는 강아지들은 비교적 ...


In [26]:
review_df.rename(columns={"시설명": "infra_id", "리뷰": "review"}, inplace=True)
review_df.head()

,infra_id,review
0,1,자주 놀러가는 곳! 전반적으로 넓은 편이어서 좋아요. 봄에는 벚꽃도 예쁜 곳이예요
1,1,그늘이 있어서 쉴수도 있고 대형견 소형견들 분리가 되어 있어 안전합니다\n강아지들이...
2,1,크기는 보라매공원 사이즈에 비해 생각보다 아담한 듯. 깨끗하고 좋음.
3,1,강아지가 근처만 가도 놀이터로 돌진합니다 ㅋㅋㅋㅋㅋ
4,1,다른 강아지들하고 교류하며 사회성 기르기 좋아요. 여기 들어오는 강아지들은 비교적 ...


In [28]:
sql = "INSERT INTO review (infra_id, comment) VALUES (%s, %s)"
for i, row in tqdm(review_df.iterrows()):
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    conn.commit()


(1, '자주 놀러가는 곳! 전반적으로 넓은 편이어서 좋아요. 봄에는 벚꽃도 예쁜 곳이예요')
(1, '그늘이 있어서 쉴수도 있고 대형견 소형견들 분리가 되어 있어 안전합니다\n강아지들이 많아 사회성 키우기에도 좋고 항상 활성화되어 있어 좋습니다')
(1, '크기는 보라매공원 사이즈에 비해 생각보다 아담한 듯. 깨끗하고 좋음.')
(1, '강아지가 근처만 가도 놀이터로 돌진합니다 ㅋㅋㅋㅋㅋ')
(1, '다른 강아지들하고 교류하며 사회성 기르기 좋아요. 여기 들어오는 강아지들은 비교적 다른 강아지들하고 잘 지내는 것 같아요. 소형견/대형견 나눠져 있어 체구에 맞게 어울려 놀 수 있어요.')
(2, '도심속의 넓은 공원으로 개설된지 수십년이 지나 나무들도 크고 동물원, 식물원, 지금도 명칭이 동일한지 모르지만 청룡열차를 비롯한 놀이기구 등이 여전하다. 주변에 사는 사람들에게는 쉽게 접근할 수 있으니 그야말로 소중한 공원이겠다.')
(2, '주차는 동물원인지 정문말고 다른데 했다. 가족 단위 방문객이 많아서 유모차 끌고다니기 편하다. 오랜만에 동물 봐서 재밌다. 동물원 설명도 아이들 수준에 맞춰져있어서 좋았다.')
(2, '금요일에 갔는데 줄 안서고 이용했어요 직원분들도 다 친절하시고 너무 무섭다 싶은 놀이기구는 없어서 잘 용했습니당 ^^( 회전그네는 생각보다 많이 어지러워요 ) 어릴때 방문했다가 성인되고 방문하니 옛추억도 생각나고 좋았습니당 동물원이랑 식물원은 무료관람 가능해요! 방문일자 (2024년 11월 1일)')
(2, '서울 도심 속에 자리잡은 오랜 역사를 자랑하는 유원지 겸 공원.\n옛 유적지 위에 지어져 공원 한쪽에 유강원이 있다.\n나무들도 울창하고 녹지도 상당하여 돌아다니면 제법 시간이 걸리는데다 동식물원, 음악분수, 놀이동산 등 다양한 즐길 거리가 준비되어 있어 가족이나 연인이 함께 가면 매우 알찬 시간을 보낼 수 있다.\n공원 가운데 팔각당은 전망대 역할을 하며 이 건물 안에도 아이들이 놀 수 있는 놀이방 같은 공간이 있다.\n공원 남쪽에는 야외

(3, '서울시 마포구 성산동 서울월드컵경기장 맞은편의 평화의 공원. 공원 내 분수도 있고, 호수도 있음. 넓은주차장 덕분에 여름,겨울을 제외하면 많은이들이 찾는 휴식공간.')
(3, '블루 마라톤 참석하느라 오랜만에 월드컵 공원에 갔습니다. 언제와도 너무 좋은 공원이에요~')
(3, '가까이 살아서 자주 방문하는데 축복 받았다고 생각이 들지경~^^\n한여름에도 그늘과 테이블이 있어서 걷고 쉬었다 올수 있고\n수고하시는분들 덕에 눈도 행복합니다\n말이 필요 없고 가보셔야되는데...\n곧 4월 중순엔 왕벚꽃 보러 오세요')
(3, '월드컵공원은 서울 마포구 상암동과 성산동 일원에\xa02002 FIFA 월드컵을 기념하여 조성된 공원입니다.\n올림픽공원(송파구),\xa0서울숲(성동구),\xa0북서울꿈의숲(강북구)과 함께 서울의 대표적인 공원입니다.\n월드컵공원은 대표 공원인 평화의공원을 비롯해 하늘공원, 노을공원, 난지천공원, 난지한강공원의 5개 테마공원으로 조성되었습니다. 현재 4개의 테마공원은 서부공원녹지사업소에서 관리하고 있으며 난지한강공원은 한강사업본부에서 관리하고 있다.')
(5, '시민을 위한 다양한 편의 시설과 , 농구, 배구 , 테니스장이 있어요. 세월의 흐름에 따라 외국의 유명한 공원 느낌도 나요.')


(5, '피자 조각 모양의 공원 이에요. 가벼운 산책을 하기에 괜찮습니다. 공원 중간에, 찻길이 있어서 횡단보도로 건너야 되는데, 기둥에 있는 버튼을 눌러야만, 보행자 파란불이 들어와요 (그것을 몰라서, 한참을 기다렸네요 ㅠㅠ)\n\n공원 중앙에, 윤봉길 의사 기념관이 있고, 남쪽으로는, 대한항공 희생자 위령탑과 삼풍 백화점 참사 위령탑이 있습니다. 삼풍 위령탑 위치가, 연관없는 위치에 있어서, 왠지 씁쓸한 마음이 들더라고요. (그래도, 대한민국 역사상 최악의 참사일 텐데... 삼풍 참사 전시관은, 더 이해할 수 없는, 경기도 포천에 위치...)')
(5, '양재천과 경부고속도로가 교차하는 곳에 두개의 숲 공원이 있다. 고속도로의 동편에 있는 매헌시민의숲은 양재시민의숲에서 윤봉길 의사의 호로 근래에 변경하였다. 조성된지 오래되어 나무들이 우거지고 크고 굵다. 윤봉길 의사의 동상과 기념관, 체육시설, 산책로와 화단 등을 잘 조성해 놓아 시민들의 휴식 공간으로 좋다.')
(5, '도심 속 여유를 즐기기 충분한 곳.\n사계절의 변화를 뚜렷하게 감상 할 수 있고 잘 꾸며 놓은 숲과 정원 운동시설 뭐하나 빠짐 없이 휴일의 여유를 만끽할 수 있는 곳')


(5, '윤봉길의사기념관이 생겼네요♡ 이름도 매헌 시민의숲으로 바뀌었어요.\n\n윤봉길의사의 호가 매헌이라, 신분당선 전철역 이름 보면서 매헌시민의숲역이 왠지 낯설지 않다고 생각했더니 역시 윤봉길의사와 연관된 장소네요.\n\n이번에 일본여행 가면서 오사카성 제4사령부 형무소 터에도 다녀오고, 가나자와 윤봉길의사 암장지와기념탑에도 다녀왔습니다.\n\n오사카와 가나자와에선 많이 슬펐는데, 그래도 선조들의 희생에 힘입어 지금의 대한민국이 있다는 생각에 힘이 납니다.\n\n문화의 힘을 가진 대한민국을 위해 후대들에게 보탬이 되는 세상을 만들어 가야겠다는 다짐을 한번더 하게 됩니다.\n\n늘 감사합니다. 잊지 않겠습니다.')
(6, '몇년을 비가오나 눈이오나 매일 산책겸 운동하러가는 멋진 공원입니다.\n편하게 걸어다닐수있고 계절마다 꽃들이 가지가지 피어서 저는 항상 사진에 담는것도 즐겁고 각종 운동기구들도 맘껏 이용할수있어  아주 저에게는 항상 고마운 공원입니다 . 그리고 무엇보다가 구석구석 깨끗하게 유지되는게 기분을 상쾌하게 해줍니다 . "공원 관리자님들께 감사드립니다 !"')
(6, '창동 초안산 옛전에는 배밭과 달동네 구옥집들이 있었는데 지금은 잘정리가\n되어서 옛모습은 추억속으로 깨끗이 잘정리 되어있으며 주위에 텃밭도 있으며 초안산 근린공원 으로서 잘갖추어 졌네요\n하늘꽃공원 쪽으로 불암산 나비공원 과같이 까페가 있으면 구민소득도 되며 초안산이 서울시에서 홍보가 될것이라 생각이되네요 ~~~')
(6, '가까운곳에 반려견 놀이터가 있어서 넘조아요 산책로도 넘좋고 자주가야할듯합니다')


(6, '아이들이 놀기에 아주좋습니다.\n근데 화장실은 좋진않습니다.')
(6, '가장 높은곳의 전망대에서 일몰을 감상하기 좋습니다. 남쪽에는 공유 텃밭이 있어 주말에 텃밭을 가꾸는 사람들을 볼 수 있습니다.')
(7, '천연 그대로의 느낌이 아직 남아있는 큰천! 둘레길 잘되어 있어 산책하기 좋아요')
(7, '주변 산책로와 자전거탈수있는길이 상당히 잘 되어있음.')
(7, '탄천유수지내 각종체육시설이 있어서 편리하게 이용할수 있네요')
(7, '음악들으면서 자연과 함께..잠시 힐링하거나 생각정리하기 좋아요')


...

(3368, '- 주상복합건물 9층에 있는 고양이카페. 덕분에 주차하는 것은 편리함. 2시간 주차 가능. 주인장님이 블로그에 오픈 여부를 공지함. 어린이는 동시입장 인원이 제한돼 있어서 아이를 데려가면 기다려야 할 수도 있음.\n\n- 2021.4 입장료 9천원. 캔 음료수만 제공하는 것으로 바뀌었음. 입장료 결제하면 음료수 or 고양이 간식 1개 제공. 식사, 와플 등 음식은 판매하지 않음.')
(1389, '몇일전에 갔는데 가성비도 진짜 엄청 좋아요 고양이 막 보고 만지고 에다가 음료수 까지 8천원인데..;; 안가시 겠어요?!')
(1389, '처음으로 고양이 카페를 방문했는데 이곳의 고양이들은 야생에서 구조된 길 잃은 고양이들이고, 스트레스를 받고, 더럽고, 귀와 눈의 일부가 없어졌기 때문에 공포증을 느꼈습니다. 그리고 가장 중요한 것은 그들이 만지는 것을 원하지 않는다는 것입니다. 익숙해지면 등을 돌리고 만질 수 있게 해줄 것입니다. 하지만 그게 전부입니다. 손톱이 다듬어지지 않아 긁히거나 감염/질병에 걸릴 위험이 있습니다. 문을 열자마자 풍기는 지독한 악취에 머리가 몹시 아팠습니다. 전반적으로 끔찍한 경험.')
(1389, '')
(1395, '애교많고 귀염뽀짝 멍멍이들이 있어요\n친절한 주인아저씨도 있구요\n바로바로 응가도치워주시고 편안하고 댕댕이들 뛰어놀기.넓은장소로 자주 이용할것같아요\n또 놀러갈께용')
(1395, '지하라서 고민하다 가까워서 갔음\n큰개 두마리와 소형견 네마리가 주인분이 기르는듯 했고 개들은 다 순했다\n사장님도 친절하셨고 입장료도 저렴한 듯\n성인 8천 초등5천원에 캔음료나 캔커피가 포함됨\n음료제조는 안하시는 듯\n시설은..\n벤치의 상태는 꽝이었음..\n개냄새가 안나고 강한향이 났는데 세제냄새인건지 방향제 인건지..\n초딩들과 갔는데 아이들은 개에게 빠졌고\n두시간가까이 있다 온 듯\n시간제한은 없어보임')
(1395, '남자분이 친절하셨다. 바닥은 좀 갈라졌지만 강아지가 좋아해서 가끔 가고 음료는 개선했으면 좋겠다. 캔

(3323, '주말 낮에는 안가봤지만 주말, 평일 저녁은 사람 없어서 좋음..')
(3323, '고양이 많습니다. 네다섯마리 유치시켜놓고 고양이카페다 우기는 카페보다 좋네요. 근데 사나운 요주의 고양이가 있지만 미리 어떤 앤지 고지해주니 조심하시면 됩니다.\n그리고 고양이는 먼저 다가가든 겁나 시간들여 기다리든 둘중 하납니다. 안놀아준다고 너무 섭섭ㄴㄴ')
(1464, '사장님들도 너무 친절하시고 애들도 너무 착해서 너무 좋은거 같아요!!! 시설도 깨끗하고이용하기 좋은거 같아요!!!')
(1464, '애견카페예요 지하1층에 있지만 사장님들께서 워낙 깔끔하셔서 깨끗하고 애카특유의 냄새라던지 그런것도 없고 사람입장료는 있지만 음료는 꼭 시켜야하는건 아니구요 당연하지만 외부음식 반입안됩니다. 상주견도 있고 믿고 호텔이나 놀이터이용하는 강아지들이 있는만큼 신뢰도도 높은편이예요^^사람 식사류도 있고요 애견들한테 간식제공은 금지예요 잘못하면 피바람이 날수도 있으니까요^^')
(1464, '입장료 6천원(필수)\n음료수(선택)\n사장님이 가게를 깔끔하게 관리하십니다. 처음갔는데 개들이 막 몰려와줘서 좋았어요 ㅋㅋㅋㅋ')
(1464, '요즘 자주 가는 신림 애견카페 ^^#')
(1464, '여기 좋아요^^ 꾸준히 관리하셔서 냄새도 안나고 쾌적하고 강아지들도 귀엽습니다')
(1467, '도베르만 받아주는 곳이 여기 뿐이라서 오게됬지만\n직원 분들도 친절하고 너무 좋아요 종종 가겠습니더')
(1467, '애견 카페는 처음인데 너무 귀여운 강아지들 덕에 힐링되고 좋았습니다 :)')
(1467, '관리가 너무 안되구요,,, 대형견 끊임 없이 짖고 적은 공간에 너무 많은 개들이 갇혀있는 느낌이었습니다.')
(1467, '사장님 및 직원분들이 친절하고 강아지친구들이 매우 귀여움')


(1467, '개들이 너무 많고 관리가 안된다는 느낌을 받았어요. 선불권은 구매 시 안내 없었는데 쓰려고 보니 유효기간이 있었다고 소멸됩니다. 구매 시  잘 알아보고 조심하세요.')
(3387, '고양이 아주 많아요\n하도 음식을 많이 줘서 통통해요. 음식을 사서 계속 주는게 고양이 건강에 좋을지 모르겠어요. 한 30% 고양이들은 한시간 내내 조금도 움직이지 않고 잠만 자요.\n아이들이 좋아해 가자 해서 간건데 고양이는 주로 어른들에게 가지 애들에겐 잘 안가요. 좀더 반기고 애교있는 고양이면 좋을텐데 하도 만지는 사람들도 많고 놀아주는 사람들도 많아 고양이들이 스포일 된거 같아요.')
(3387, '너무 예쁘고 관리 잘된 야옹이들~ 이 사랑스러운 아이들이 가지고 있는 아픔들이 조금이나마ㅈ위안이 되어줬음 좋겠어요 사장님도 많은 아가들 돌보시느라 힘드실텐데 감사합니다~^-^')
(3387, '나눠진공간에 공간도 잘 꾸며 놓았네요 냥이들은 약 20마리쯤 되구요 노령묘부터 아기냥이까지 다양한 연령대로 편안한 분위기에 잘 어울리고 있어요 지금은 입장료 9천원에 1시간 30분이용이에요 잠깐 쉬어가기 좋습니다^^ 동물을 좋아하시는 분들은 참고하시길~')
(3387, '일반 고양이 카페가 아닌 유기묘 카페라고 해서 갔는데 생각보다 더 좋은 공간이었습니다\n특히 인상적이었던건 베란다가 천장만 뚫린 거였어요!\n냥이랑 같이 햇빛과 바람 맞으며 앉아 있을 수 있다는 게 진짜 집사가 되어본 느낌이라 행복했어요\n사진으로 못남긴 아늑한 공간도 또 베란다 옆에 있어서 꽤 넓어요')
(3387, '고양이도 너무 귀엽고 너무 좋아요~매장도 넓고 청결하고 고양이가 엄청 건강하고 생기가 넘처보이네요\n츄르 가격은 2,000원 하는데 꽤 괜찮아요')
(3388, '고양이 숫자가 많습니다.\n여러 종이 있고 모두 다른 고양이 카페 대비해서는 온순한 편입니다.(성깔 있는 개체들도 한 둘 있음)\n다만 전체적으로 고양이들 성정이 부드러운 편이라 어린아이의 우악스러운 손길에도 잘 참아주는 편이며(다만

2503it [00:52, 47.25it/s]

(3388, '너무 귀여워요 다들…\n외국인 분들도 많이 오시네요\n고양이는 야행성이고 하루의 대부분을 잔다는 사실을 간과하고 가셔야 합니다\n낮잠시간이라 다 자는게 아니고 고양이는 원래 야행성입니다')
(3390, '해외에서 한국 방문했다가 고양이가 보고싶어 방문했어요. 리뷰보고 물어 물어가길 참 잘했어요. 사장님께서 전화통화로 길안내도 친절히 잘 해주셨고 방문했을 때도 따뜻하게 맞아주셨어요. 열아홉마리 고양이들도 넘 순둥순둥 예쁘고 카페도 깨끗하고 분위기도 참 좋아서 또 가고 싶어요. 한국에서 만든 가장 특별한 추억 중 하나가 될거에요. 고양리라 좋은 날 카페 이름대로 딱 맞네요. 감사해요!')
(3390, '20여마리의 고양이천국, 고양이라 좋은 날.\n\n여타 반려동물까페와 다르게 정말 건강하게 관리된 개냥이들만 모여있다.\n\n평일: 음료포함 1만원(사실상 무제한)\n주말: 가격동일, 90분 시간제한\n\n90분이면 지치고도 남는다. 단순 개냥이를 너머 이불보만 깔아주면 와서 머리를 먼저 대고 눕는다.\n\n애교 만점, 활동성 만점의 다양한 냥이들의 집합소\n\n러시안 블루인 블루부터 뽀뽀 등 종류도 다양하고 사납지 않아 냥이초심자도 편한 모습으로 만날 수 있는 곳.')
(3390, '고양이들이 매우 건강하고 편안해보였어요. 주인분께서도 아끼시는게 느껴졌습니다. 냥이들이 만지는걸 좋아하지 않는 눈치라 구경만 했어요. 바라만보아도 행복하게 해주는 냥냥이들 \U0001fa77')
(3390, '여기 진짜 좋아요!\n고양이 카페는 처음인데\n이렇게 좋을줄 몰랐어요.\n엄청 깨끗하고 냥들도 프랜들리해요!')
(3390, '고양이들이 엄청 많아요. 사람들도 끊임없이 들어오는데 고양이들이 너무 사랑스럽고 이뻐서 보다보면 시간 가는 줄 모르겠더라고요.')
(3400, '고양이들이 많아서 시간 가는 줄 모르고 놀았어요! 사장님두 친절하시구 냥이들도 다 애교가 흘러 넘쳐요\U0001fa75 내부도 깔끔하고 음료도 맛있어요 ❤️')
(3400, '아이가 너무너무 행복해

In [29]:
conn.close()